In [53]:
import os
from ultralytics import YOLO
import pandas as pd
from tqdm import tqdm

In [54]:
DIR = '../data/patients'
patient_folders = sorted([os.path.join(DIR, x, 'images') for x in os.listdir(DIR) if os.path.isdir(os.path.join(DIR, x))])

In [55]:
sorted(patient_folders)

['../data/patients/patient 01/images',
 '../data/patients/patient 02/images',
 '../data/patients/patient 03/images',
 '../data/patients/patient 04/images',
 '../data/patients/patient 05/images',
 '../data/patients/patient 06/images',
 '../data/patients/patient 07/images',
 '../data/patients/patient 08/images',
 '../data/patients/patient 09/images',
 '../data/patients/patient 10/images']

In [56]:
model = YOLO('weights.pt')
CONF = 0.532

In [57]:
results = []

for patient_folder in tqdm(patient_folders):
    image_files = os.listdir(patient_folder)
    patient = int(patient_folder.split('/')[-2].split(' ')[-1])
    plasma_cells = 0
    non_plasma_cells = 0
    
    for image_file in image_files:
        detections = model(os.path.join(patient_folder, image_file), conf=CONF, verbose=False)[0].boxes.cls

        plasma_cells += int((detections == 0).sum())
        non_plasma_cells += int((detections == 1).sum())

    total = plasma_cells + non_plasma_cells    
    percentage = round(plasma_cells / total, 3)
    diagnosis = 'diseased' if percentage >= 0.1 else 'normal'

    results.append([patient, plasma_cells, non_plasma_cells, total, percentage, diagnosis])

100%|██████████| 10/10 [00:32<00:00,  3.29s/it]


In [58]:
ground_truth = pd.read_csv('../data/patients/diagnosis.csv')
ground_truth

,patient,plasma_cells,non_plasma_cells,total,percentage,diagnosis
0,1,82,120,202,0.406,diseased
1,2,78,127,205,0.374,diseased
2,3,72,128,200,0.360,diseased
3,4,82,116,198,0.414,diseased
4,5,71,129,200,0.355,diseased
5,6,15,185,200,0.075,normal
6,7,18,187,205,0.088,normal
7,8,9,199,208,0.043,normal
8,9,14,188,202,0.069,normal
9,10,15,188,203,0.074,normal


In [59]:
results = pd.DataFrame(results, columns=['patient', 'plasma_cells', 'non_plasma_cell', 'total', 'percentage', 'diagnosis'])
results

,patient,plasma_cells,non_plasma_cell,total,percentage,diagnosis
0,1,77,119,196,0.393,diseased
1,2,83,121,204,0.407,diseased
2,3,77,124,201,0.383,diseased
3,4,71,99,170,0.418,diseased
4,5,70,108,178,0.393,diseased
5,6,18,197,215,0.084,normal
6,7,26,208,234,0.111,diseased
7,8,15,238,253,0.059,normal
8,9,15,209,224,0.067,normal
9,10,14,197,211,0.066,normal
